# Hierarchy

Python code can be ordered using the following hierarchy.
Note that some hierarchy levels are not applicable for Jupyter Notebooks but only for "normal" Python code.

## Packages [optional]
A directory conaining .py files (i.e. Modules) and/ or subpackages. A directory becomes a Python Package if the file 

    __init__.py 
    
(which is usually empty) is present.

## Modules
A file containing Python code. It may be top level or inside a Package. A module may contain functions, classes, variables (usually constants) and/ or code.
    
Code inside a Module is automatically executed if the module is imported. Code which should be only executed if the module itself is executed (not by imports from other modules) is protected using the following check:
    
    if __name__ == '__main__':
        code
        
Note that it is perfectly fine in Python to have constants and functions directly in Modules, outside classes (unlike e.g. in Java, where everything must be inside a class). The module-level namespaces avoid naming conflicts.

# Imports

In [2]:
%cat ../code/my_module.py # example module to be imported

"""example file for module imports"""

__version__ = '0.0.1'

my_secret = 'you_will_never_guess'
my_list = [1, 2, 3, 4]
my_tuple = ('a', 'b', 'c')

def double_me(x):
    """doubles input value"""
    return 2*x

def power_me(x, i: int, times_two=False):
    """useless example function"""
    mult = 2 if times_two else 1
    return mult*x**i

class Multiplier:
    """class that creates multiplier objects with a given factor"""
    def __init__(self, i):
        self.i = i
    def get_result(self, x):
        """returns factor specified with initialization multiplied by input value"""
        return self.i*x

mult2 = Multiplier(2)
mult3 = Multiplier(3)

print('my_module imported')

if __name__ == '__main__':
    # this code is not executed when importing this module, but only if the module is executed
    print('executing main block')


## Pythonpath

In [3]:
import sys
import os

In [4]:
new_path = os.path.abspath('../code')
if new_path not in sys.path:
    sys.path.append(new_path)

Python searches for modules in import statements in the following places:
* Current directory
* Python builtins
* Directories in Pythonpath

The code above checks if the given directory is included in Pythonpath and adds it if not.

This pattern is also useful e.g. for unit tests if the test cases are kept in a directory different from the code.

## Import Specific Objects

In [5]:
from my_module import double_me
from my_module import my_list as my_constants # rename object with importings

my_module imported


The 

    from module import object [as alias]
    
pattern imports specific objects of the module in local namespace.

Note that the print statement at the bottom of the module is executed, although it is not in one of the imported objects. Python runs the complete module script the first time it is used in any import.

In [6]:
double_me(42)

84

In [7]:
my_constants

[1, 2, 3, 4]

#### Antipatterns

In [8]:
my_constants.append(5)
my_constants

[1, 2, 3, 4, 5]

Extremely dangerous: changing (mutable) data in an other module. All imports referring to this data will be affected.

In [9]:
my_tuple = ('this', 'is', 'very', 'important')
my_tuple

('this', 'is', 'very', 'important')

In [10]:
from my_module import * # Imports all members of the module into local namespace

In [11]:
my_tuple

('a', 'b', 'c')

The usage of the pattern

    from module import *

has the following drawbacks and is therefore not recommended:
* There could be naming conflicts between local and imported objects
* There is no information or control in the module where the import statement has been executed which object originates from which module.

## Import Whole Module as Separate Namespace

In [12]:
import my_module

This is the recommended method to import modules, especially if multiple objects of the module are required. If the number of imported objects is small, the *from module import object* syntax is also fine.

The code in the imported module is only executed once during the first import. Thus the print statement is not shown again.

In [13]:
my_module.double_me(42)

84

In [14]:
mult7 = my_module.Multiplier(7)
mult7.get_result(7)

49

In [15]:
my_module.mult2.get_result(3)

6

In [16]:
my_module.my_list # note the effect of my_constants.append(5)

[1, 2, 3, 4, 5]

In [17]:
my_module.my_list is my_constants # both are pointers to the same list object

True

Note that the modification of the imported list changes this list anywhere it is imported (actually, all imports refer to the same object). 
Therefore it is very dangerous to import and modify mutable data structures from other modules.

## Import from Packages

In [18]:
import mypackage # note that this does not import submodules in this package

In [19]:
try:
    mypackage.my_module2.triple_me(3)
except AttributeError as e:
    print(e)

module 'mypackage' has no attribute 'my_module2'


In [20]:
import mypackage.my_module2 # explicit import of modules in package into package namespace

In [21]:
mypackage.my_module2.triple_me(3)

9

In [22]:
from mypackage import my_module2 # import submodules into current namespace

In [23]:
my_module2.triple_me(3)

9

In [24]:
from mypackage.my_module2 import triple_me as tripler
# import objects from submodules into current namespace

In [25]:
tripler(4)

12

In [26]:
from mypackage import * # imports only content/imports of __init__.py and modules
# listed in the __all__ variable in the init file into current namespace

In [27]:
%cat ../example_files/mypackage/__init__.py # note that my_module4 is included here, but not my_module3

cat: ../example_files/mypackage/__init__.py: No such file or directory


In [28]:
try:
    my_module3.square_me(4) # not imported with from package import *
except NameError as e:
    print(e)

name 'my_module3' is not defined


In [29]:
my_module4.divide_me_by_2(8) # imported with from package import * because module name
# is in __all__ of __init__.py

4.0

# Main Block

In some cases, the same Python module id used both in imports and executed directly.
In this case, a main block can be defined which is not executed during import. There is no pre-defined method to do this in Python (like the main class in Java), but the following code pattern is used:

    if __name__ == '__main__':
        do_something
        


In [30]:
import my_module # note that the print command inside the main block is not executed 

In [31]:
exec(open('../code/my_module.py').read()) # antipattern: never use this as part of a program workflow
# use import instead

my_module imported
executing main block


# Inspection

## Docstrings

In [32]:
my_module.__doc__

'example file for module imports'

In [33]:
my_module.double_me.__doc__

'doubles input value'

In [34]:
my_module.Multiplier.__doc__

'class that creates multiplier objects with a given factor'

In [35]:
my_module.Multiplier.get_result.__doc__

'returns factor specified with initialization multiplied by input value'

Docstrings, i.e. strings that are defined immediately after start of modules, functions, classes or methods, are not only helpful for reading code. They can be accessed on runtime with the doc dunder property.

## Help Function

In [36]:
help(my_module)

Help on module my_module:

NAME
    my_module - example file for module imports

CLASSES
    builtins.object
        Multiplier
    
    class Multiplier(builtins.object)
     |  Multiplier(i)
     |  
     |  class that creates multiplier objects with a given factor
     |  
     |  Methods defined here:
     |  
     |  __init__(self, i)
     |      Initialize self.  See help(type(self)) for accurate signature.
     |  
     |  get_result(self, x)
     |      returns factor specified with initialization multiplied by input value
     |  
     |  ----------------------------------------------------------------------
     |  Data descriptors defined here:
     |  
     |  __dict__
     |      dictionary for instance variables (if defined)
     |  
     |  __weakref__
     |      list of weak references to the object (if defined)

FUNCTIONS
    double_me(x)
        doubles input value
    
    power_me(x, i: int, times_two=False)
        useless example function

DATA
    mult2 = <my_mo

In [37]:
help(my_module.Multiplier)

Help on class Multiplier in module my_module:

class Multiplier(builtins.object)
 |  Multiplier(i)
 |  
 |  class that creates multiplier objects with a given factor
 |  
 |  Methods defined here:
 |  
 |  __init__(self, i)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  get_result(self, x)
 |      returns factor specified with initialization multiplied by input value
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



The builtin help function automatically generates information about the objects using the signatures and docstrings.

## Inspect Module

In [38]:
import inspect

In [39]:
inspect.getmembers(my_module.double_me)

[('__annotations__', {}),
 ('__call__',
  <method-wrapper '__call__' of function object at 0x7f03f0f78d08>),
 ('__class__', function),
 ('__closure__', None),
 ('__code__',
  <code object double_me at 0x7f03f0f6f660, file "/home/jovyan/Tutorial/code/my_module.py", line 9>),
 ('__defaults__', None),
 ('__delattr__',
  <method-wrapper '__delattr__' of function object at 0x7f03f0f78d08>),
 ('__dict__', {}),
 ('__dir__', <function function.__dir__()>),
 ('__doc__', 'doubles input value'),
 ('__eq__', <method-wrapper '__eq__' of function object at 0x7f03f0f78d08>),
 ('__format__', <function function.__format__(format_spec, /)>),
 ('__ge__', <method-wrapper '__ge__' of function object at 0x7f03f0f78d08>),
 ('__get__', <method-wrapper '__get__' of function object at 0x7f03f0f78d08>),
 ('__getattribute__',
  <method-wrapper '__getattribute__' of function object at 0x7f03f0f78d08>),
 ('__globals__',
  {'__name__': 'my_module',
   '__doc__': 'example file for module imports',
   '__package__': '

Gives (very long) list of all available objects inside the stated object.

In [40]:
inspect.getdoc(my_module.double_me)

'doubles input value'

Docstring - equivalent to 

    my_module.double_me.__doc__

In [49]:
inspect.getsource(my_module.double_me)

'def double_me(x):\n    """doubles input value"""\n    return 2*x\n'

shows source code of object

In [64]:
module = inspect.getmodule(my_module.double_me)
module

<module 'my_module' from '/home/jovyan/Tutorial/code/my_module.py'>

Yields the module of the stated object. Note that this is a reference to the actual module, not just its name and path as text.

In [61]:
sig = inspect.signature(my_module.power_me)
sig.parameters

mappingproxy({'x': <Parameter "x">,
              'i': <Parameter "i: int">,
              'times_two': <Parameter "times_two=False">})

Shows arguments (including type hints and standard values) of stated function / method

In [65]:
'i' in sig.parameters

True

Check if function has a specific parameter

In [46]:
inspect.getfullargspec(my_module.power_me)

FullArgSpec(args=['x', 'i', 'times_two'], varargs=None, varkw=None, defaults=(False,), kwonlyargs=[], kwonlydefaults=None, annotations={'i': <class 'int'>})

Gets additional information about parameters.